# Libs upload

In [2]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import numpy as np
import seaborn as sns

pd.set_option("display.precision", 3)
pd.set_option('display.float_format', lambda x: '%.5f' % x)
pd.set_option('display.max_rows', 4)
pd.set_option('display.max_columns', 15)

In [3]:
def well_display_logs(dataset, wellname, fmname, net_var1, net_var2, ref_depth, depth_step, print_flag, path):
    data = dataset[(dataset.well==wellname) & (dataset.FORMATION_up == fmname)]
    depth = data['TST']
    grn = data['GR_N']
    vsh = data['VSH']
    rhob = data['RHOB'] 
    npss = data['NPSS']
    rdeep = data['RDEEP']
    phit = data['PHIT']
    net_init = data[net_var1]*100 
    net = data[net_var2]*100
    calc = data['calc']*10
    perm = data['LPERM']
    perm1 = data['PERM_DS']
    perm2 = data['PERM_US']

    net_init_sum = (net_init.sum()*0.001).round(1)
    net_sum = (net.sum()*0.001).round(1)

    calcite_sum = data['calc'].sum()
    calcite_sum = (calcite_sum*0.1).round(1)

    fig, ax = plt.subplots(1,4, figsize=(7,11), sharey=True)
    well_bal_tops = dataset[(dataset.well == wellname)].groupby('FORMATION')[ref_depth].apply(lambda x: x.iloc[0]).reset_index()
    ax[0].yaxis.set_ticks(np.arange(min(depth), max(depth), depth_step))
    ax[0].plot(grn, depth, color='lightgreen', lw=3, zorder=10)
    ax[0].invert_yaxis() 
    ax[0].set_xlim(0, 150) 
    for i in well_bal_tops[well_bal_tops.FORMATION.str.contains(fmname)].FORMATION:
        ax[0].hlines(well_bal_tops[well_bal_tops.FORMATION==i][ref_depth].iloc[0], 
                    xmin=0, xmax=1000, linewidth=2, color='black', lw=2, alpha=0.33)
    ax[0].set_title(net_var1)
    twin0 = ax[0].twiny()
    twin0.plot(vsh, depth, color='black', alpha=0.33, zorder=5)
    twin0.vlines(0.5, ymin=min(depth), ymax=max(depth), color='black', linestyle='dashed', linewidth=1)
    twin0.xaxis.set_ticks(np.arange(0, 1.5, 0.5))
    twin0.set_xlim(0, 1.5)
    twin0.grid(True)
    twin0.plot(net_init, depth, color='orange', linewidth=0.5, alpha=1)
    twin0.fill_betweenx(depth,net_init, color='orange', alpha=0.25)
    ax[1].plot(rhob, depth, color='red', alpha=0.75) 
    ax[1].invert_yaxis() 
    ax[1].xaxis.set_ticks(np.arange(1.65, 2.65, 0.3))
    ax[1].grid(True)
    ax[1].set_xlim(1.65, 2.65)
    for i in well_bal_tops[well_bal_tops.FORMATION.str.contains(fmname)].FORMATION:
        ax[1].hlines(well_bal_tops[well_bal_tops.FORMATION==i][ref_depth].iloc[0], 
        xmin=0, xmax=150, linewidth=2, color='black', lw=2, alpha=0.33)
        ax[1].text(1.67, well_bal_tops[well_bal_tops.FORMATION==i][ref_depth].iloc[0]+0.5*depth_step, i, fontsize = 8, color ="black")
    twin1 = ax[1].twiny()
    twin1.plot(npss, depth, color='blue', alpha=0.75)
    twin1.plot(calc, depth, color='#6ccbf3', linewidth=0.1)
    twin1.fill_betweenx(depth,calc, color='#6ccbf3', alpha=0.5)
    twin1.set_xlim(0.6, 0)
    twin1.grid(False)
    ax[2].plot(phit, depth, color='green', linestyle='dashed')
    ax[2].set_xlim(0.3, 0)
    ax[2].xaxis.set_ticks(np.arange(0, 0.3, 0.1))
    ax[2].tick_params(labelsize=10)
    ax[2].grid(True)
    ax[2].invert_yaxis()
    for i in well_bal_tops[well_bal_tops.FORMATION.str.contains(fmname)].FORMATION:
        ax[2].hlines(well_bal_tops[well_bal_tops.FORMATION==i][ref_depth].iloc[0], xmin=0, xmax=1000, linewidth=2, color='black', lw=2, alpha=0.33)
    ax[2].vlines(0.13, ymin=min(depth), ymax=max(depth), color='black', linestyle='dashed', linewidth=1)
    ax[2].set_title(net_var2)
    twin2 = ax[2].twiny()
    twin2.plot(net, depth, color='orange', linewidth=0.5, alpha=1)
    twin2.fill_betweenx(depth,net, color='orange', alpha=0.33)
    twin2.set_xlim(0, 1)
    twin2.set_ylim(min(depth), max(depth))
    twin2.xaxis.set_ticks([])
    twin2.grid(False)
    twin3 = ax[2].twiny()
    twin3.plot(rdeep, depth, color='black', linewidth=1, alpha=1)
    twin3.set_xscale('log')
    twin3.set_xlim(1, 200)
    twin3.grid(False)
    ax[3].plot(perm, depth, color='purple', alpha=0.75)
    ax[3].plot(perm1, depth, color='purple', alpha=0.5)
    ax[3].plot(perm2, depth, color='purple', alpha=0.5)
    ax[3].set_xscale('log')
    ax[3].set_xlim(0.1, 1000)
    ax[3].invert_yaxis()
    for i in well_bal_tops[well_bal_tops.FORMATION.str.contains(fmname)].FORMATION:
        ax[3].hlines(well_bal_tops[well_bal_tops.FORMATION==i][ref_depth].iloc[0], xmin=0, xmax=1000, linewidth=2, color='black', lw=2, alpha=0.33)
    fig.suptitle(wellname + ' ' + fmname + ' ' + ref_depth + 
                 ' net_vsh: ' + str(net_init_sum) + 'm' 
                 ' net: ' + str(net_sum) + 'm' 
                 ' calc: ' + str(calcite_sum),
                 fontsize=14)
    fig.tight_layout()
    if print_flag == 'print':
        fig.savefig(path + wellname + '.png', dpi=300)
    else:
        pass
    plt.show()

# well_display_logs(bal8_v2, 'C33Z', 'Balakhany VIII', 'NET_VSH', 'NET', 'TST', 5, 'dontprint', 'C:\\jupyter\\SPP\\output\\')

# Bal8 upload

In [4]:
df_bal8_azr_vshclp2 = pd.read_csv('C:\jupyter\SPP\inputoutput\general_logs\df_bal8_azr_vshclp2_calc.csv').drop('Unnamed: 0', axis=1)
df_bal8_azr_vshclp2 = df_bal8_azr_vshclp2[~df_bal8_azr_vshclp2.well.isin(['E20Y', 'E10Z', 'J13X'])]
well_phit_flag8 = df_bal8_azr_vshclp2[df_bal8_azr_vshclp2.phit_flag==1].groupby('well')['phit_flag'].apply(lambda x: x.iloc[0]).reset_index().well.unique()
df_bal8_azr_vshclp2_flag = df_bal8_azr_vshclp2[df_bal8_azr_vshclp2.well.isin(well_phit_flag8)]

df_bal8_azr_v2_init = pd.read_csv('C:\jupyter\SPP\inputoutput\general_logs\df_bal8_azr_v2.csv').drop('Unnamed: 0', axis=1)
df_bal8_azr_v2 = df_bal8_azr_v2_init.set_index(['well','TST']).join(
                df_bal8_azr_vshclp2.set_index(['well','TST'])[['calc','calc_net']]
                ).reset_index()

well_phit_flag8 = df_bal8_azr_v2[df_bal8_azr_v2.phit_flag==1].groupby('well')['phit_flag'].apply(lambda x: x.iloc[0]).reset_index().well.unique()
df_bal8_azr_v2_flag = df_bal8_azr_v2[df_bal8_azr_v2.well.isin(well_phit_flag8)]
bal8_v2 = df_bal8_azr_v2.copy()
bal8_phit_v2 = df_bal8_azr_v2_flag.copy()

ntd8 = pd.read_csv(r'C:\jupyter\SPP\inputoutput\layers\ntd_top_phi_bot8_bp.csv').drop('Unnamed: 0', axis=1)

In [5]:
gr_cube = pd.read_csv(r'C:\jupyter\SPP\inputoutput\raw\gr_cube_md_interp.csv').drop('Unnamed: 0', axis=1)
bal8_phit_v2_cube = bal8_phit_v2.set_index(['well','MD']).join(gr_cube.set_index(['well','MD'])).reset_index()

## Bal8 calculation

In [6]:
black_list8 = ['C18X', 'D20Y', 'B09', 'B09', 'D26', 'B18', 'D01Z', 'C03', 'C05', 'E10Z', 'E20Y', 'J13X']
bal8_v2 = bal8_v2[~bal8_v2.well.isin(black_list8)]
bal8_v2 = bal8_v2[bal8_v2.FORMATION != 'Balakhany VIII']
bal8_phit_v2 = bal8_phit_v2[~bal8_phit_v2.well.isin(black_list8)]
bal8_phit_v2 = bal8_phit_v2[bal8_phit_v2.FORMATION != 'Balakhany VIII']
# bal8_v2.loc[bal8_v2.well=='C27Z', 'NET'] = bal8_v2[bal8_v2.well=='C27Z']['NET_VSH']
# bal8_v2.loc[bal8_v2.well=='D21', 'NET'] = bal8_v2[bal8_v2.well=='D21']['NET_VSH']
# bal8_v2.loc[(bal8_v2.well=='B17') & (bal8_v2.VSH > 0.4), 'NET_VSH'] = 0
# bal8_v2.loc[bal8_v2.well=='D19Y', 'NET'] = bal8_v2[bal8_v2.well=='D19Y']['NET_VSH']
# bal8_v2.loc[bal8_v2.well=='C27X', 'NET'] = bal8_v2[bal8_v2.well=='C27X']['NET_VSH']

net_bal8 = bal8_v2.groupby('well')[['NET','NET_VSH','Xmean','Ymean','TVD_SCS','phit_flag', 'calc']].agg(
    {'NET': lambda x: x.sum()*0.1, 
     'NET_VSH': lambda x: x.sum()*0.1, 
     'Xmean': lambda x: x.iloc[0], 
     'Ymean': lambda x: x.iloc[0],
     'TVD_SCS': 'mean', 
     'phit_flag': lambda x: x.iloc[0],
     'calc': lambda x: x.sum()*0.1}
    ).reset_index()
net_phit_bal8 = bal8_phit_v2.groupby('well')[['NET','NET_VSH','Xmean','Ymean', 'TVD_SCS','phit_flag','calc']].agg(
    {'NET': lambda x: x.sum()*0.1, 
     'NET_VSH': lambda x: x.sum()*0.1, 
     'Xmean': lambda x: x.iloc[0], 
     'Ymean': lambda x: x.iloc[0],
     'TVD_SCS': 'mean',  
     'phit_flag': lambda x: x.iloc[0],
     'calc': lambda x: x.sum()*0.1}
    ).reset_index()

In [ ]:
net_phit_bal8[net_phit_bal8.calc > 1]

## Bal8 xplot

In [ ]:
net_phit_bal8['diff_netvsh_net'] = net_phit_bal8.NET_VSH - net_phit_bal8.NET
net_phit_bal8['diff_netvsh_net_rel'] = net_phit_bal8.diff_netvsh_net/net_phit_bal8.NET_VSH
net_phit_bal8['FORMATION_up'] = 'Balakhany VIII'

def scatter_plot8(net_bal8, net_phit_bal8, calc_sum, coef):
    net_vsh = go.Scatter(x=net_bal8.NET_VSH, 
                    y=net_bal8.NET, 
                    mode='markers', 
                    marker=dict(size=10, 
                                color='#12ca1b',
                                opacity=0.75, 
                                line=dict(width=0.75, color='black')),
                    customdata = net_bal8[['well', 'NET', 'NET_VSH']].round(1),
                    hovertemplate="".join(["well:%{customdata[0]}," +
                                        "net:%{customdata[1]}," +
                                        "net_vsh:%{customdata[2]}<extra></extra>"]),
                    )
    net = go.Scatter(x=net_phit_bal8.NET_VSH, 
                    y=net_phit_bal8.NET, 
                    mode='markers', 
                    marker=dict(size=10, 
                                color='#f09319',
                                opacity=0.75, 
                                line=dict(width=0.75, color='black')),
                    customdata = net_phit_bal8[['well', 'NET', 'NET_VSH']].round(1),
                    hovertemplate="".join(["well:%{customdata[0]}," +
                                        "net:%{customdata[1]}," +
                                        "net_vsh:%{customdata[2]}<extra></extra>"]),
                    )
    calc = go.Scatter(x=net_phit_bal8[net_phit_bal8.calc > calc_sum].NET_VSH, 
                y=net_phit_bal8[net_phit_bal8.calc > calc_sum].NET, 
                mode='markers', 
                marker=dict(size=10, 
                            color='#05b2c9',
                            opacity=0.75, 
                            line=dict(width=0.75, color='gray')),
                customdata = net_phit_bal8[net_phit_bal8.calc > calc_sum][['well', 'NET', 'NET_VSH', 'calc']].round(1),
                hovertemplate="".join(["well:%{customdata[0]}," +
                                       "net:%{customdata[1]}," +
                                       "net_vsh:%{customdata[2]}," +
                                       "calc:%{customdata[3]}<extra></extra>"])
                )
    axis_range = [20, 100]
    line = go.Scatter(  x=[axis_range[0], axis_range[1]], y=[axis_range[0], axis_range[1]], 
                        mode='lines', 
                        line=dict(width=2, color='red'),
                        showlegend=False)
    coef_perc = coef/100 
    line_up = go.Scatter(   x=[axis_range[0], axis_range[1]], y=[axis_range[0]*(1+coef_perc), axis_range[1]*(1+coef_perc)], 
                            mode='lines', 
                            line=dict(width=2, color='red', dash='dash'),
                            showlegend=False)
    line_down = go.Scatter( x=[axis_range[0], axis_range[1]], y=[axis_range[0]*(1-coef_perc), axis_range[1]*(1-coef_perc)], 
                            mode='lines', 
                            line=dict(width=2, color='red', dash='dash'),
                            showlegend=False)
    fig = go.Figure()
    # fig.add_trace(net_vsh)
    fig.add_trace(net)
    fig.add_trace(calc)
    fig.add_trace(line_up)
    fig.add_trace(line)
    fig.add_trace(line_down)
    fig.update_xaxes(title_text='NET_VSH', range=[axis_range[0], axis_range[1]])
    fig.update_yaxes(title_text='NET', range=[axis_range[0], axis_range[1]])
    fig.update_layout(  title='NET vs NET_VSH Bal8' + ' coef: ' + str(coef) + '%',
                        margin=dict(l=10, r=10, t=50, b=10),
                        autosize=False,
                        width=800,
                        height=600,
                        showlegend=False)
    fig.show()
    # fig.write_html(r'C:\jupyter\SPP\plots\netvsh_net_bal8.html')

scatter_plot8(net_bal8, net_phit_bal8, 1, coef=10)

In [9]:
# well_display_logs(bal8_v2, 'D20Y', 'Balakhany X', 'NET_VSH', 'NET', 'TST', 5, 'dontprint', 'C:\\jupyter\\SPP\\output\\')
# well_display_logs(bal8_v2, 'C27Z', 'Balakhany VIII', 'NET_VSH', 'NET', 'TST', 5, 'dontprint', 'C:\\jupyter\\SPP\\output\\')
# well_display_logs(bal8_v2, 'B36', 'Balakhany VIII', 'NET_VSH', 'NET', 'TST', 5, 'dontprint', 'C:\\jupyter\\SPP\\output\\')
# well_display_logs(bal8_v2, 'D02Y', 'Balakhany VIII', 'NET_VSH', 'NET', 'TST', 5, 'dontprint', 'C:\\jupyter\\SPP\\output\\')
# well_display_logs(bal8_v2, 'C34', 'Balakhany VIII', 'NET_VSH', 'NET', 'TST', 5, 'dontprint', 'C:\\jupyter\\SPP\\output\\')
# well_display_logs(bal8_v2, 'B29', 'Balakhany VIII', 'NET_VSH', 'NET', 'TST', 5, 'dontprint', 'C:\\jupyter\\SPP\\output\\')

# Bal10 upload

In [10]:
df_bal10_vshclp2 = pd.read_csv('C:\jupyter\SPP\inputoutput\general_logs\df_bal10_vshclp2_calc.csv').drop('Unnamed: 0', axis=1)
well_phit_flag10 = df_bal10_vshclp2[df_bal10_vshclp2.phit_flag==1].groupby('well')['phit_flag'].apply(lambda x: x.iloc[0]).reset_index().well.unique()
df_bal10_vshclp2_flag = df_bal10_vshclp2[df_bal10_vshclp2.well.isin(well_phit_flag10)]

bal10_v2 = df_bal10_vshclp2.copy()
bal10_phit_v2 = df_bal10_vshclp2_flag.copy()

ntd10 = pd.read_csv(r'C:\jupyter\SPP\inputoutput\layers\ntd_top_phi_bot10_bp.csv').drop('Unnamed: 0', axis=1)

## Bal10 calculation

In [11]:
net_bal10 = bal10_v2.groupby('well')[['NET','NET_VSH','Xmean','Ymean','TVD_SCS','phit_flag', 'calc']].agg(
    {'NET': lambda x: x.sum()*0.1, 
     'NET_VSH': lambda x: x.sum()*0.1, 
     'Xmean': lambda x: x.iloc[0], 
     'Ymean': lambda x: x.iloc[0], 
     'TVD_SCS': 'mean',
     'phit_flag': lambda x: x.iloc[0],
     'calc': lambda x: x.sum()*0.1}
    ).reset_index()
net_phit_bal10 = bal10_phit_v2.groupby('well')[['NET','NET_VSH','Xmean','Ymean','TVD_SCS','phit_flag','calc']].agg(
    {'NET': lambda x: x.sum()*0.1, 
     'NET_VSH': lambda x: x.sum()*0.1, 
     'Xmean': lambda x: x.iloc[0], 
     'Ymean': lambda x: x.iloc[0],
     'TVD_SCS': 'mean', 
     'phit_flag': lambda x: x.iloc[0],
     'calc': lambda x: x.sum()*0.1}
    ).reset_index()

## Bal10 xplot

In [ ]:
net_phit_bal10['diff_netvsh_net'] = net_phit_bal10.NET_VSH - net_phit_bal10.NET
net_phit_bal10['diff_netvsh_net_rel'] = net_phit_bal10.diff_netvsh_net/net_phit_bal10.NET_VSH
net_phit_bal10['FORMATION_up'] = 'Balakhany X'

def scatter_plot10(net_bal10, net_phit_bal10, calc_sum, coef):
    net_vsh = go.Scatter(   x=net_bal10.NET_VSH, 
                            y=net_bal10.NET, 
                            mode='markers', 
                            marker=dict(size=10, 
                                        color='#12ca1b',
                                        opacity=0.75, 
                                        line=dict(width=0.75, color='black')),
                            customdata = net_bal10[['well', 'NET', 'NET_VSH']].round(1),
                            hovertemplate="".join(["well:%{customdata[0]}," +
                                                "net:%{customdata[1]}," +
                                                "net_vsh:%{customdata[2]}<extra></extra>"]),
                            )
    net = go.Scatter(   x=net_phit_bal10.NET_VSH, 
                        y=net_phit_bal10.NET, 
                        mode='markers', 
                        marker=dict(size=10, 
                                    color='#f09319',
                                    opacity=0.75, 
                                    line=dict(width=0.75, color='black')),
                        customdata = net_phit_bal10[['well', 'NET', 'NET_VSH']].round(1),
                        hovertemplate="".join(["well:%{customdata[0]}," +
                                            "net:%{customdata[1]}," +
                                            "net_vsh:%{customdata[2]}<extra></extra>"]),
                        )
    calc = go.Scatter(  x=net_phit_bal10[net_phit_bal10.calc > calc_sum].NET_VSH, 
                        y=net_phit_bal10[net_phit_bal10.calc > calc_sum].NET, 
                        mode='markers', 
                        marker=dict(size=10, 
                                    color='#05b2c9',
                                    opacity=0.75, 
                                    line=dict(width=0.75, color='gray')),
                        customdata = net_phit_bal10[net_phit_bal10.calc > calc_sum][['well', 'NET', 'NET_VSH', 'calc']].round(1),
                        hovertemplate="".join(["well:%{customdata[0]}," +
                                            "net:%{customdata[1]}," +
                                            "net_vsh:%{customdata[2]}," +
                                            "calc:%{customdata[3]}<extra></extra>"])
                        )
    axis_range = [0, 85]
    line = go.Scatter(  x=[axis_range[0], axis_range[1]], y=[axis_range[0], axis_range[1]], 
                        mode='lines', 
                        line=dict(width=2, color='red'),
                        showlegend=False)
    coef_perc = coef/100 
    line_up = go.Scatter(   x=[axis_range[0], axis_range[1]], y=[axis_range[0]*(1+coef_perc), axis_range[1]*(1+coef_perc)], 
                            mode='lines', 
                            line=dict(width=2, color='red', dash='dash'),
                            showlegend=False)
    line_down = go.Scatter( x=[axis_range[0], axis_range[1]], y=[axis_range[0]*(1-coef_perc), axis_range[1]*(1-coef_perc)], 
                            mode='lines', 
                            line=dict(width=2, color='red', dash='dash'),
                            showlegend=False)
    fig = go.Figure()
    # fig.add_trace(net_vsh)
    fig.add_trace(net)
    fig.add_trace(calc)
    fig.add_trace(line_up)
    fig.add_trace(line)
    fig.add_trace(line_down)
    fig.update_xaxes(title_text='NET_VSH', range=[axis_range[0], axis_range[1]])
    fig.update_yaxes(title_text='NET', range=[axis_range[0], axis_range[1]])
    fig.update_layout(  title='NET vs NET_VSH Bal10' + ' coef: ' + str(coef) + '%',
                        margin=dict(l=10, r=10, t=50, b=10),
                        autosize=False,
                        width=800,
                        height=600,
                        showlegend=False)
    fig.show()
    # fig.write_html(r'C:\jupyter\SPP\plots\netvsh_net_bal10.html')

scatter_plot10(net_bal10, net_phit_bal10, 0.5, coef=10)

In [13]:
# well_display_logs(bal10_v2, 'A12YST1', 'Balakhany X', 'NET_VSH', 'NET', 'TST', 5, 'dontprint', 'C:\\jupyter\\SPP\\output\\')
# well_display_logs(bal10_v2, 'C33Z', 'Balakhany X', 'NET_VSH', 'NET', 'TST', 5, 'dontprint', 'C:\\jupyter\\SPP\\output\\')
# well_display_logs(bal10_v2, 'E01', 'Balakhany X', 'NET_VSH', 'NET', 'TST', 5, 'dontprint', 'C:\\jupyter\\SPP\\output\\')
# well_display_logs(bal10_v2, 'H01Y', 'Balakhany X', 'NET_VSH', 'NET', 'TST', 5, 'dontprint', 'C:\\jupyter\\SPP\\output\\')
# well_display_logs(bal10_v2, 'E11Z', 'Balakhany X', 'NET_VSH', 'NET', 'TST', 5, 'dontprint', 'C:\\jupyter\\SPP\\output\\')
# well_display_logs(bal10_v2, 'A01X', 'Balakhany X', 'NET_VSH', 'NET', 'TST', 5, 'dontprint', 'C:\\jupyter\\SPP\\output\\')
# well_display_logs(bal10_v2, 'A04', 'Balakhany X', 'NET_VSH', 'NET', 'TST', 5, 'dontprint', 'C:\\jupyter\\SPP\\output\\')
# well_display_logs(bal10_v2, 'A12W', 'Balakhany X', 'NET_VSH', 'NET', 'TST', 5, 'dontprint', 'C:\\jupyter\\SPP\\output\\')
# well_display_logs(bal10_v2, 'A08Z', 'Balakhany X', 'NET_VSH', 'NET', 'TST', 5, 'dontprint', 'C:\\jupyter\\SPP\\output\\')
# well_display_logs(bal10_v2, 'A10X', 'Balakhany X', 'NET_VSH', 'NET', 'TST', 5, 'dontprint', 'C:\\jupyter\\SPP\\output\\')
# well_display_logs(bal10_v2, 'A10', 'Balakhany X', 'NET_VSH', 'NET', 'TST', 5, 'dontprint', 'C:\\jupyter\\SPP\\output\\')
# well_display_logs(bal10_v2, 'A13W', 'Balakhany X', 'NET_VSH', 'NET', 'TST', 5, 'dontprint', 'C:\\jupyter\\SPP\\output\\')
# well_display_logs(bal10_v2, 'C27Z', 'Balakhany X', 'NET_VSH', 'NET', 'TST', 5, 'dontprint', 'C:\\jupyter\\SPP\\output\\')
# well_display_logs(bal10_v2, 'A09Y', 'Balakhany X', 'NET_VSH', 'NET', 'TST', 5, 'dontprint', 'C:\\jupyter\\SPP\\output\\')
# well_display_logs(bal10_v2, 'C18', 'Balakhany X', 'NET_VSH', 'NET', 'TST', 5, 'dontprint', 'C:\\jupyter\\SPP\\output\\')
# well_display_logs(bal10_v2, 'A07', 'Balakhany X', 'NET_VSH', 'NET', 'TST', 5, 'dontprint', 'C:\\jupyter\\SPP\\output\\')

## Diff netvsh vs net map

In [ ]:
diff_netvsh_net = net_phit_bal10[net_phit_bal10.diff_netvsh_net_rel > 0.15]
diff_netvsh_net = diff_netvsh_net[diff_netvsh_net.TVD_SCS.notna()]
diff_netvsh_net['TVD_SCS'] = diff_netvsh_net['TVD_SCS'].astype(int)

def net_reductin_display(net_phit_bal10, diff_netvsh_net, calc_sum, width_sel=1200, height_sel=800):
    calcite_points = go.Scatter(    x=net_phit_bal10[net_phit_bal10.calc > calc_sum]['Xmean'], 
                                    y=net_phit_bal10[net_phit_bal10.calc > calc_sum]['Ymean'], 
                                    mode='markers', 
                                    marker=dict(
                                                color = '#11c9e9',
                                                opacity=0.5,
                                                size=net_phit_bal10[net_phit_bal10.calc > calc_sum]['calc'], 
                                                sizemode='area', 
                                                sizeref=2.*max(net_phit_bal10[net_phit_bal10.calc > calc_sum]['calc'])/(20.**2), 
                                                sizemin=4,
                                                line=dict(width=1, color='black')),
                                    customdata = net_phit_bal10[net_phit_bal10.calc > calc_sum][['well', 'calc']].round(1),
                                    hovertemplate="".join(["well:%{customdata[0]}, calc %{customdata[1]}<extra></extra>"])
                                )
    diff_net_points = go.Scatter(   x=diff_netvsh_net['Xmean'], 
                                    y=diff_netvsh_net['Ymean'], 
                                    mode='markers', 
                                    marker=dict(
                                                color = '#e79d13',
                                                opacity=0.5,
                                                size=diff_netvsh_net['diff_netvsh_net'], 
                                                sizemode='area', 
                                                sizeref=4.*max(diff_netvsh_net['diff_netvsh_net'])/(40.**2), 
                                                sizemin=4,
                                                line=dict(width=1, color='black')),
                                    customdata = diff_netvsh_net[['well', 'NET', 'NET_VSH', 'diff_netvsh_net', 'TVD_SCS']].round(2),
                                    hovertemplate="".join([ "well %{customdata[0]}," +
                                                            "net %{customdata[1]}," +
                                                            "netvsh %{customdata[2]}," +
                                                            "diff %{customdata[3]}," +
                                                            "tvd %{customdata[4]}<extra></extra>"])
                                    )
    well_points = go.Scatter(   x=net_phit_bal10['Xmean'], 
                                y=net_phit_bal10['Ymean'], 
                                mode='markers',
                                marker=dict(symbol='cross',
                                            color='black',
                                            size=3,
                                            ),
                                customdata = net_phit_bal10[['well','TVD_SCS']].round(0),
                                hovertemplate="".join(["well %{customdata[0]}" +
                                                       "tvd %{customdata[1]}<extra></extra>"])
                                ) 
    fig = go.Figure()  
    # fig.add_trace(calcite_points)
    fig.add_trace(diff_net_points)
    fig.add_trace(well_points)
    fig.update_layout(  
                        # autosize=True,
                        autosize=False,
                        width=width_sel,
                        height=height_sel,
                        margin=dict(l=10, r=10, b=10, t=50),
                        showlegend=False,
                        title = 'Net_vsh vs net reduction Bal10, calcite > 1%',
                        title_font = dict(size=20)
                        )
    fig.show()
    # fig.write_html(r'C:\jupyter\SPP\plots\netvsh_net_map_bal10.html')

net_reductin_display(net_phit_bal10, diff_netvsh_net, calc_sum=1, width_sel=1400, height_sel=800)

In [15]:
net_phit_bal_total = pd.concat([net_phit_bal8, net_phit_bal10], axis=0)
net_phit_bal_total = net_phit_bal_total[~net_phit_bal_total.TVD_SCS.isna()]
# net_phit_bal_total.to_csv(r'C:\jupyter\SPP\inputoutput\netvsh_net\net_phit_bal_total.csv')